# NOTE

This notebook is just an example to demonstrate D1 compatibility with the `TimeXer` model. Considering that there is no concrete design for a TSLib specific D2 layer, for the time being we are using the `EncoderDecoderDataModule` and `BaseModel` for implementing `TimeXer`. The implementation is rather confusing with many overlapping bits because the D2 isn't solely built for TSLib, but is a demonstration of how `TimeXer` works with the latest version of v2 rework and shows promise for more models from TSlib.

In [1]:
%pip install pytorch-forecasting



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from typing import Any, Dict, List, Optional, Tuple, Union

from lightning.pytorch import Trainer
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import Dataset

In [13]:
from pytorch_forecasting.data.data_module import EncoderDecoderTimeSeriesDataModule
from pytorch_forecasting.data.encoders import (
    EncoderNormalizer,
    NaNLabelEncoder,
    TorchNormalizer,
)
from pytorch_forecasting.data.timeseries import TimeSeries
from pytorch_forecasting.metrics import MAE, SMAPE
from pytorch_forecasting.models.timexer._timexer import TimeXer

In [14]:
num_series = 100
seq_length = 50
data_list = []
for i in range(num_series):
    x = np.arange(seq_length)
    y = np.sin(x / 5.0) + np.random.normal(scale=0.1, size=seq_length)
    category = i % 5
    static_value = np.random.rand()
    for t in range(seq_length - 1):
        data_list.append(
            {
                "series_id": i,
                "time_idx": t,
                "x": y[t],
                "y": y[t + 1],
                "static_feature": static_value,
            }
        )
data_df = pd.DataFrame(data_list)
data_df.head()

,series_id,time_idx,x,y,static_feature
0,0,0,-0.014910,0.060817,0.388177
1,0,1,0.060817,0.363816,0.388177
2,0,2,0.363816,0.585283,0.388177
3,0,3,0.585283,0.675532,0.388177
4,0,4,0.675532,0.731210,0.388177


In [5]:
dataset = TimeSeries(
    data=data_df,
    time="time_idx",
    target="y",
    group=["series_id"],
    num=["x", "static_feature"],
)

/home/pranav/Desktop/code/pytorch-forecasting/pytorch_forecasting/data/timeseries/_timeseries_v2.py:106: UserWarning: TimeSeries is part of an experimental rework of the pytorch-forecasting data layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. For beta testing, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warnings.warn(


In [6]:
data_module = EncoderDecoderTimeSeriesDataModule(
    time_series_dataset=dataset,
    max_encoder_length=30,
    max_prediction_length=1,
    batch_size=16,
    scalers={"x": StandardScaler(), "static_feature": StandardScaler()},
    target_normalizer=TorchNormalizer(),
)

In [7]:
data_module.setup(stage="fit")
data_module.metadata

{'encoder_cat': 0,
 'encoder_cont': 2,
 'decoder_cat': 0,
 'decoder_cont': 0,
 'target': 1,
 'static_categorical_features': 0,
 'static_continuous_features': 0,
 'max_encoder_length': 30,
 'max_prediction_length': 1,
 'min_encoder_length': 30,
 'min_prediction_length': 1}

In [8]:
model = TimeXer(
    loss=nn.L1Loss(),
    logging_metrics=[MAE(), SMAPE()],
    context_length=30,
    prediction_length=1,
    task_name="long_term_forecast",
    features="MS",
    d_model=32,
    n_heads=2,
    e_layers=1,
    d_ff=64,
    dropout=0.1,
    patch_length=1,
    use_norm=False,
    metadata=data_module.metadata,
    optimizer="adam",
    optimizer_params={"lr": 1e-3},
)

In [9]:
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=10,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 6GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                   | Params | Mode 
----------------------------------------------------------------
0 | loss         | L1Loss                 | 0      | train
1 | en_embedding | EnEmbedding            | 64     | train
2 | ex_embedding | DataEmbedding_inverted | 992    | train
3 | encoder      | Encoder                | 12.9 K | train
4 | head         | FlattenHead            | 993    | train
----------------------------------------------------------------
14.9 K    Trainable params
0         Non-trainable params
14.9 K    Total params
0.060     Tot

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect resul

Training: |          | 0/? [00:00<?, ?it/s]

/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([

...

/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([

Validation: |          | 0/? [00:00<?, ?it/s]

/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([

Validation: |          | 0/? [00:00<?, ?it/s]

/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/pranav/Desktop/code/pytorch-forecasting/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([